In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
# 품목기준코드를 저장할 리스트
list_code = []

TOTAL_PAGE = 7223
pbar = tqdm(total=TOTAL_PAGE)
page = 1

while page < TOTAL_PAGE:
    try:
        # 의약품안전나라 제품 검색 url
        url = 'https://nedrug.mfds.go.kr/searchDrug?page=' + str(page)
        # html 문서에서 table 읽기
        table = pd.read_html(url, encoding='utf-8')

        # table에서 품목키준코드 column만 추출
        df_code = table[0]['품목기준코드']

        # 정규표현식을 이용하여 코드 앞에 있는 '품목기준코드' 문자열 제거
        for i in df_code:
            code = i.replace('품목기준코드', '')
            list_code.append(code)
        pbar.update(1)
        page += 1
    except:
        continue
        
pbar.close() 

100%|█████████████████████████████████████| 7223/7223 [1:18:13<00:00,  1.54it/s]


In [9]:
# 품목기준코드 텍스트 파일에 저장
f = open("code.txt", 'w')
for code in list_code:
    data = code + ","
    f.write(data)
f.close()

In [2]:
# 텍스트 파일에 저장한 품목기준코드 불러오기
f = open('code.txt', 'r')
list_code = f.read().split(',')
list_code

['200511454',
 '199806988',
 '200105441',
 '200708707',
 '201100396',
 '200710604',
 '200710605',
 '202002854',
 '200804293',
 '201005181',
 '201500185',
 '201502755',
 '201506752',
 '201508324',
 '200908818',
 '200800961',
 '201100370',
 '201605830',
 '201206516',
 '201602137',
 '202002598',
 '201602366',
 '202003594',
 '201907165',
 '200106909',
 '201908440',
 '201908439',
 '201101738',
 '201908181',
 '201006073',
 '201007520',
 '200905737',
 '201802921',
 '202003612',
 '202005681',
 '202000967',
 '202004442',
 '202001992',
 '202006178',
 '201402407',
 '201308886',
 '201601598',
 '201401168',
 '202007109',
 '202001933',
 '201803218',
 '201308394',
 '202001104',
 '202004307',
 '201804720',
 '202001946',
 '201903701',
 '202003483',
 '201506474',
 '201001965',
 '201201035',
 '201402737',
 '201308682',
 '201101104',
 '201109606',
 '202001156',
 '202001426',
 '201207230',
 '202001501',
 '201006877',
 '202003338',
 '201506504',
 '201805199',
 '201404321',
 '202005962',
 '202102965',
 '2010

In [3]:
data = []
row = {}
i = 0
NUM = 8000 

pbar = tqdm(total=NUM)

while i < NUM:
    code = list_code[i]
    # 상세 보기 url
    url_detail = 'https://nedrug.mfds.go.kr/pbp/CCBBB01/getItemDetail?itemSeq=' + code
    try:
        # html 문서에서 table 읽기
        table_detail = pd.read_html(url_detail, encoding='utf-8', index_col=0)[0].T
        drug_name = table_detail['제품명'][1]
        shape = table_detail['성상'][1]
        company_name = table_detail['업체명'][1]
    except:
        print("read_html error")
        continue
        
    try:    
        res_detail = requests.get(url_detail)
    except:
        print("GET error")
        continue
        
    if res_detail.status_code == 200:
        soup = BeautifulSoup(res_detail.content, 'html.parser')

        usage = ''

        for div in soup.find_all('div', class_='info_box mt20 pt0'):
            usage += re.sub('\n', '', div.text)

        ingred = []

        for data_ingred in soup.find_all('p', class_='note'):
            name_ingred, amount_ingred, unit_ingred = '', '', ''
            for row_ingred in re.split('\s\s\|\s\s',data_ingred.text):
                if '성분명' in row_ingred:
                    name_ingred = re.split('\s\:\s', row_ingred)[1]
                elif '분량' in row_ingred:
                    amount_ingred = re.split('\s\:\s', row_ingred)[1]
                elif '단위' in row_ingred:
                    unit_ingred = re.split('\s\:\s', row_ingred)[1]
            if [name_ingred, amount_ingred, unit_ingred] not in ingred:
                ingred.append([name_ingred, amount_ingred, unit_ingred])

        caution = []        

        for row_caution in soup.find_all('p', class_='title'):
            caution.append(re.sub('\s', '', row_caution.text))

        if len(caution) == 0:
            caution='[#None]'
        if ingred == [['', '', '']]:
            ingred='[#None]'

        data.append({'제품코드': 'kdrug-'+code, '제품명':drug_name, '제조사': company_name, '성상':shape, '용법용량':usage, '주의사항':re.sub("'", "", str(caution)[1:-1]), '성분': str(ingred)[1:-1]})
        pbar.update(1)
        i+=1
    else:
        print('response code is not 200')
        continue

pbar.close()

  2%|▊                                     | 163/8000 [01:29<1:34:51,  1.38it/s]

read_html error


  9%|███▎                                  | 700/8000 [08:35<1:55:47,  1.05it/s]

GET error


 11%|████▎                                 | 911/8000 [11:05<1:05:43,  1.80it/s]

...

 98%|████████████████████████████████████▏| 7815/8000 [1:42:27<02:02,  1.51it/s]

read_html error


 99%|████████████████████████████████████▌| 7916/8000 [1:43:45<00:37,  2.25it/s]

read_html error


100%|█████████████████████████████████████| 8000/8000 [1:44:55<00:00,  1.27it/s]


In [ ]:
data = []
row = {}
i = 8000
NUM = 16000 

pbar = tqdm(total=i)

while i < NUM:
    code = list_code[i]
    # 상세 보기 url
    url_detail = 'https://nedrug.mfds.go.kr/pbp/CCBBB01/getItemDetail?itemSeq=' + code
    try:
        # html 문서에서 table 읽기
        table_detail = pd.read_html(url_detail, encoding='utf-8', index_col=0)[0].T
        drug_name = table_detail['제품명'][1]
        shape = table_detail['성상'][1]
        company_name = table_detail['업체명'][1]
    except:
        print("read_html error")
        continue
        
    try:    
        res_detail = requests.get(url_detail)
    except:
        print("GET error")
        continue
        
    if res_detail.status_code == 200:
        soup = BeautifulSoup(res_detail.content, 'html.parser')

        usage = ''

        for div in soup.find_all('div', class_='info_box mt20 pt0'):
            usage += re.sub('\n', '', div.text)

        ingred = []

        for data_ingred in soup.find_all('p', class_='note'):
            name_ingred, amount_ingred, unit_ingred = '', '', ''
            for row_ingred in re.split('\s\s\|\s\s',data_ingred.text):
                if '성분명' in row_ingred:
                    name_ingred = re.split('\s\:\s', row_ingred)[1]
                elif '분량' in row_ingred:
                    amount_ingred = re.split('\s\:\s', row_ingred)[1]
                elif '단위' in row_ingred:
                    unit_ingred = re.split('\s\:\s', row_ingred)[1]
            if [name_ingred, amount_ingred, unit_ingred] not in ingred:
                ingred.append([name_ingred, amount_ingred, unit_ingred])

        caution = []        

        for row_caution in soup.find_all('p', class_='title'):
            caution.append(re.sub('\s', '', row_caution.text))

        if len(caution) == 0:
            caution='[#None]'
        if ingred == [['', '', '']]:
            ingred='[#None]'

        data.append({'제품코드': 'kdrug-'+code, '제품명':drug_name, '제조사': company_name, '성상':shape, '용법용량':usage, '주의사항':re.sub("'", "", str(caution)[1:-1]), '성분': str(ingred)[1:-1]})
        pbar.update(1)
        i+=1
    else:
        print('response code is not 200')
        continue

pbar.close()

 52%|█████████████████▎               | 4187/8000 [1:58:15<102:26:18, 96.72s/it]

GET error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error


 75%|███████████████████████████▊         | 6018/8000 [3:27:29<47:03,  1.42s/it]

read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_htm

read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_htm

...

read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_htm

read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_htm

read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_htm

read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_html error
read_htm

 76%|██████████████████████████▌        | 6061/8000 [3:39:24<1:10:23,  2.18s/it]

In [8]:
columns = ['제품코드', '제품명', '제조사', '성상', '용법용량', '주의사항', '성분']    
df = pd.DataFrame(data=data, columns=columns)
# df.to_csv('result(1).csv', index=False, encoding='utf-8-sig')
df.to_csv('result(2).csv', index=False, encoding='utf-8-sig')

# 코드 설명

In [5]:
# 의약품안전나라 제품 검색 url
url = 'https://nedrug.mfds.go.kr/searchDrug'
# html 문서에서 table 읽기
table = pd.read_html(url, encoding='utf-8')
# 첫 번째 테이블
table[0]

,순번,제품명,업체명,품목기준코드,허가번호,허가일,품목구분,취소/취하,취소/취하일자,주성분,...,품목분류,전문의약품,완제/원료구분,허가/신고,제조/수입,수입제조국,마약구분,신약구분,표준코드,ATC코드
0,1,제품명 #4500제이라스틱,업체명(주)곰스포츠,품목기준코드200511454,허가번호4,허가일2005-01-27,품목구분의약외품,취소/취하구분정상,취소/취하일자,주성분,...,품목분류 반창고(33800),전문의약품 의약외품,완제/원료구분,허가/신고허가,제조/수입수입,수입제조국,마약구분-,신약구분 N,표준코드,ATC코드-
1,2,제품명 (주)미성종합가스의료용산소,업체명(주)미성종합가스,품목기준코드199806988,허가번호1,허가일1998-07-13,품목구분의약품,취소/취하구분정상,취소/취하일자,주성분산소,...,품목분류,전문의약품 전문의약품,완제/원료구분완제의약품,허가/신고신고,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드,ATC코드-
2,3,제품명 (주)창신의료용산소,업체명(주)창신,품목기준코드200105441,허가번호1,허가일2001-09-27,품목구분의약품,취소/취하구분정상,취소/취하일자,주성분산소,...,품목분류,전문의약품 전문의약품,완제/원료구분완제의약품,허가/신고신고,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드,ATC코드-
3,4,제품명 (트리플라이프)-그나랜면생리대,업체명(주)트리플라이프,품목기준코드200708707,허가번호19,허가일2007-07-27,품목구분의약외품,취소/취하구분폐업,취소/취하일자2019-05-24,주성분,...,품목분류 생리대(31100),전문의약품 의약외품,완제/원료구분,허가/신고허가,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드,ATC코드-
4,5,"제품명 .덴탈크리닉2080오리지날알파블루치약 , 2.2080어드밴스후레쉬치약 , ...",업체명애경산업(주),품목기준코드201100396,허가번호5011,허가일2011-01-25,품목구분의약외품,취소/취하구분정상,취소/취하일자,"주성분토코페롤아세테이트,덴탈타입실리카,플루오르화나트륨",...,품목분류 치약제(41400),전문의약품,완제/원료구분,허가/신고신고,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드,ATC코드-
5,6,제품명 0.45%엔에이시엘.케이20주,업체명제이더블유생명과학(주),품목기준코드200710604,허가번호22,허가일2007-09-18,품목구분의약품,취소/취하구분정상,취소/취하일자,"주성분염화칼륨,염화나트륨",...,품목분류,전문의약품 전문의약품,완제/원료구분완제의약품,허가/신고신고,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드8806789000109...,ATC코드B05BB01
6,7,제품명 0.9%엔에이시엘.케이40주,업체명제이더블유생명과학(주),품목기준코드200710605,허가번호29,허가일2007-09-18,품목구분의약품,취소/취하구분정상,취소/취하일자,"주성분염화칼륨,염화나트륨",...,품목분류,전문의약품 전문의약품,완제/원료구분완제의약품,허가/신고신고,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드8806789000208...,ATC코드B05BB01
7,8,제품명 0720세니타이저액(에탄올),업체명주식회사어반비씨엘,품목기준코드202002854,허가번호5013,허가일2020-04-22,품목구분의약외품,취소/취하구분정상,취소/취하일자,주성분에탄올,...,품목분류 외용소독제(46000),전문의약품,완제/원료구분,허가/신고신고,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드,ATC코드-
8,9,제품명 1)딥퓨러티액(에탄올)2)뉴클린앤후레쉬손소독제(에탄올)3)24.7핸드세니타...,업체명유씨엘(주),품목기준코드200804293,허가번호299,허가일2008-04-18,품목구분의약외품,취소/취하구분폐업,취소/취하일자2018-12-10,주성분에탄올,...,품목분류 외용소독제(46000),전문의약품 의약외품,완제/원료구분,허가/신고허가,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드,ATC코드-
9,10,제품명 1)알펫손소독제액(에탄올)2)애니크린손소독제액(에탄올),업체명(주)크린피스,품목기준코드201005181,허가번호96,허가일2010-08-27,품목구분의약외품,취소/취하구분정상,취소/취하일자,주성분에탄올,...,품목분류 외용소독제(46000),전문의약품 의약외품,완제/원료구분,허가/신고허가,제조/수입제조,수입제조국,마약구분-,신약구분 N,표준코드,ATC코드-


In [6]:
# table에서 품목키준코드 column만 추출
df_code = table[0]['품목기준코드']
df_code

0     품목기준코드200511454
1     품목기준코드199806988
2     품목기준코드200105441
3     품목기준코드200708707
4     품목기준코드201100396
5     품목기준코드200710604
6     품목기준코드200710605
7     품목기준코드202002854
8     품목기준코드200804293
9     품목기준코드201005181
10    품목기준코드201500185
11    품목기준코드201502755
12    품목기준코드201506752
13    품목기준코드201508324
14    품목기준코드200908818
Name: 품목기준코드, dtype: object

In [7]:
# 품목기준코드를 저장할 리스트
list_code = []

# 정규표현식을 이용하여 코드 앞에 있는 '품목기준코드' 문자열 제거
for i in df_code:
    code = i.replace('품목기준코드', '')
    list_code.append(code)
    
list_code

['200511454',
 '199806988',
 '200105441',
 '200708707',
 '201100396',
 '200710604',
 '200710605',
 '202002854',
 '200804293',
 '201005181',
 '201500185',
 '201502755',
 '201506752',
 '201508324',
 '200908818']

In [8]:
# 상세 보기 url
url_detail = 'https://nedrug.mfds.go.kr/pbp/CCBBB01/getItemDetail?itemSeq=201100396'
# html 문서에서 table 읽기
table_detail = pd.read_html(url_detail, encoding='utf-8')
# 첫 번째 테이블
table_detail[0]

,0,1
0,제품명,".덴탈크리닉2080오리지날알파블루치약 , 2.2080어드밴스후레쉬치약 , 3.청은차..."
1,성상,1.덴탈크리닉2080오리지날알파블루치약 : 흰색과 청색의 줄무늬모양 페이스트제 2....
2,업체명,애경산업(주)
3,위탁제조업체,NaN
4,전문/일반,NaN
5,허가일,2011-01-25
6,품목기준코드,201100396
7,표준코드,NaN
8,분류코드,41400


In [ ]:
# 상세페이지 GET
res_detail = requests.get(url_detail)
# BeautifulSoup 객체 생성
soup = BeautifulSoup(res_detail.content, 'html.parser')

In [ ]:
ingred = []

for data_ingred in soup.find_all('p', class_='note'):
    name, amount, unit = '', '', ''
    for row_ingred in re.split('\s\s\|\s\s',data_ingred.text):
        if '성분명' in row_ingred:
            name = re.split('\s\:\s', row_ingred)[1]
        elif '분량' in row_ingred:
            amount = re.split('\s\:\s', row_ingred)[1]
        elif '단위' in row_ingred:
            unit = re.split('\s\:\s', row_ingred)[1]
    if [name, amount, unit] not in ingred:
        ingred.append([name, amount, unit])
ingred

In [9]:
caution = []        

for row_caution in soup.find_all('p', class_='title'):
    caution.append(row_caution.text)

caution

['1) 이 치약의 불소 함유량은 1000ppm임(총 함유량은 1,000ppm 이하이어야 한다.)',
 '2) 6세 이하의 어린이가 사용할 경우, 1회당 완두콩 크기 정도의 소량의 치약을 사용하고, 빨아 먹거나, 삼키지 않도록 보호자의 지도하에 사용할 것',
 '3) 6세 이하의 어린이가 많은 양을 삼켰을 경우, 즉시 의사 또는 치과의사와 상의할 것',
 '4) 6세 이하의 어린이의 손에 닿지 않는 곳에 보관할 것']